In [1]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [5]:
titantic = pd.read_csv('titanic.txt')
titantic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [11]:
print titantic.head()[['pclass', 'survived', 'age', 'embarked', 'boat', 'sex']]

  pclass  survived      age     embarked   boat     sex
0    1st         1  29.0000  Southampton      2  female
1    1st         0   2.0000  Southampton    NaN  female
2    1st         0  30.0000  Southampton  (135)    male
3    1st         0  25.0000  Southampton    NaN  female
4    1st         1   0.9167  Southampton     11    male

[5 rows x 6 columns]


In [20]:
# for any column in cols, it converts the one column to multiple columns
# for example column name - sex, values Female, Male
# it converts to two columns, sex=female, sex=male
from sklearn import feature_extraction
def one_hot_dataframe(data, cols, replace=True):
    vec = feature_extraction.DictVectorizer()
    vecData = pd.DataFrame(vec.fit_transform(data[cols].to_dict(outtype='records')).toarray())
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData)
titantic, titanic_n = one_hot_dataframe(titantic, ['pclass','embarked','sex'], replace=True)

In [21]:
titanic_n

,embarked,embarked=Cherbourg,embarked=Queenstown,embarked=Southampton,pclass=1st,pclass=2nd,pclass=3rd,sex=female,sex=male
0,0,0,0,1,1,0,0,1,0
1,0,0,0,1,1,0,0,1,0
2,0,0,0,1,1,0,0,0,1
3,0,0,0,1,1,0,0,1,0
4,0,0,0,1,1,0,0,0,1
5,0,0,0,1,1,0,0,0,1
6,0,0,0,1,1,0,0,1,0
7,0,0,0,1,1,0,0,0,1
8,0,0,0,1,1,0,0,1,0
9,0,1,0,0,1,0,0,0,1
